# Create Baseline email file - Code is adapted from Kaggle

In [ ]:
#!pip install numpy==1.22.4

In [3]:
""" RUN THIS CELL TO GET THE RIGHT FORMATTING """
import requests
from IPython.core.display import HTML
css_file = 'https://raw.githubusercontent.com/bsethwalker/clemson-cs4300/main/css/cpsc6300.css'
styles = requests.get(css_file).text
HTML(styles)

from dotenv import load_dotenv
import os
from openai import OpenAI
import time
import glob

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

from sklearn.model_selection import train_test_split
import pathlib
from sklearn.utils import shuffle


import warnings
warnings.filterwarnings('ignore')

load_dotenv()

my_id = os.getenv("OPENAI_API_KEY")

client = OpenAI(
    api_key=os.environ.get("my_id"),
)

In [4]:
dataframe = pd.read_csv("/Users/roulierk/Documents/Clemson/Thesis/Seven-Phishing-email-data/Enron.csv")
dataframe = shuffle(dataframe)
dataframe.loc[dataframe["label"]==0, "label"] = 0
dataframe.loc[dataframe["label"]==1, "label"] = 1
dataframe["label"] = dataframe["label"].astype(float)

In [5]:
#print(dataframe['label'].to_string(index=False))

In [6]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29767 entries, 8797 to 2875
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   subject  29569 non-null  object 
 1   body     29767 non-null  object 
 2   label    29767 non-null  float64
dtypes: float64(1), object(2)
memory usage: 930.2+ KB


### Reduce the dataset to 10.  Don't need all of 11k elements for this.

In [7]:
# Getting first 10 rows from dataframe 
df_10 = dataframe.head(10) 
  
#  Printing df_10 
df_10.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10 entries, 8797 to 16306
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   subject  10 non-null     object 
 1   body     10 non-null     object 
 2   label    10 non-null     float64
dtypes: float64(1), object(2)
memory usage: 320.0+ bytes


### Using the reduce dataset (10), Clean up the data, removing zeros and ones, remove "status"

In [8]:
allColumns = list(df_10.columns)
print(allColumns)

['subject', 'body', 'label']


### Add defaults to the file so that later we can play with temp, etc.

In [9]:
new_df = df_10.assign(Legit=np.NaN, Phish=np.NaN, Unknown=np.NaN,temp=0, elapsed=0)  #add the defaults for now

In [10]:
new_df

,subject,body,label,Legit,Phish,Unknown,temp,elapsed
8797,re : dragon curves ( thai baht forecast ),we have prepared the attached two reports to a...,0.0,NaN,NaN,NaN,0,0
26099,( none ),"dear , friend !\nadvertising + talent manageme...",1.0,NaN,NaN,NaN,0,0
4538,earn huge profits starting today mc ncxb ueytfw,"hello ,\nif i could show you a business where ...",1.0,NaN,NaN,NaN,0,0
11077,response to ferc : eol,attached are the responses to the questions of...,0.0,NaN,NaN,NaN,0,0
28135,"joint venture ( please , respond immediately )",from : barrister samson ajayi malolo\nphilip a...,1.0,NaN,NaN,NaN,0,0
6761,re : telephone interview with the research group,hi mike :\nthanks for responding so quickly . ...,0.0,NaN,NaN,NaN,0,0
28906,take a look at this profit - making powerhouse,wysak petroieum ( wysk )\ncurrent price : 0 . ...,1.0,NaN,NaN,NaN,0,0
21285,book administrators - gas,the highlighted names on this list are the boo...,0.0,NaN,NaN,NaN,0,0
28911,do you know how others sav on top - selling ta...,it is a chemist - site that provides better se...,1.0,NaN,NaN,NaN,0,0
16306,new services customer,"group ,\njust wanted to let you know that begi...",0.0,NaN,NaN,NaN,0,0


### Create a new "prompt" file to drive the ChatGPT.

In [11]:
new_df.to_csv('/Users/roulierk/Documents/Clemson/Thesis/Seven-Phishing-email-data/test_email_prompt1.csv', header=False, index=False, index_label=None, mode='w')  